In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


keras.utils.set_random_seed(253)

In [ ]:
class PINNSolver():
    def __init__(self, model, alpha, beta, X_r):
        self.model = model
        self.t = X_r

        self.alpha = alpha
        self.beta = beta


    def get_r(self):
        
        t = tf.convert_to_tensor(self.t, dtype='float32')
        
        with tf.GradientTape(persistent=True) as tape:

            tape.watch(t)
            u = self.model(t)
            u_t = tape.gradient(u, t)

            u_tt = tape.gradient(u_t, t)
        
        del tape

        return(u_tt + (self.alpha * u_t) + (self.beta * u))
    

    # t is inputs
    # u is actual outputs
    def loss_fn(self, t, u):

        r = self.get_r()
        phi_r = tf.reduce_mean(tf.square(r))
        
        loss = phi_r

        t_tensor = tf.convert_to_tensor(t, dtype='float32')
        u_pred = self.model(t_tensor)
        loss += tf.reduce_mean(tf.square(u - u_pred))
        
        return loss
    

    def get_grad(self, t, u):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(self.model.trainable_variables)
            loss = self.loss_fn(t, u)

        g = tape.gradient(loss, self.model.trainable_variables)
        del tape

        return loss, g
    
    def solve_with_TFoptimizer(self, optimizer: keras.optimizers.Optimizer, t, u, epochs):
        
        def train_step():
            loss, gradient = self.get_grad(t, u)
            optimizer.apply_gradients(zip(gradient, self.model.trainable_variables))
            return loss
        
        for i in range(epochs):
            loss = train_step()
            print("Epoch: " + str(i))
            print("Loss: " + str(loss))
            print("Alpha: " + str(self.alpha))
            print("Beta: " + str(self.beta))
            print("\n")
            # save every epoch so i can jump ship at any time and still have a network to show for it
            self.model.save_weights("alpha_beta_problem.weights2.h5")
            
            

In [ ]:
def yajuns_function(t: np.float32):
    return(-1 * np.power(np.e, t)) + np.power(np.e, 2*t) 

# lowercase x is x variable, Uppercase is x and t variables combined
# *_0 is initial data
# *_b is boundary data
# *_r is collocation points
# *_data is initial data, boundary data
# *_d is distributed exact solutions
# *_param is _data + _d


t_data = np.arange(0, 5, .01, dtype='float32')
u_data = np.array([yajuns_function(t) for t in t_data], dtype='float32')


In [ ]:
# make layers of the model
# most accurate 2 period wave is
# 20, 15, 10, 5 with .5 dropout in between

def make_network() -> tuple:
    layers = []         

    layers.append(keras.layers.Dense(20, activation = 'elu', input_shape = (1,)))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    layers.append(keras.layers.Dense(20, activation = 'elu'))
    output_layer = keras.layers.Dense(1, activation = 'linear')
    layers.append(output_layer)
    
    output_layer.add_weight(shape = (), dtype='float32', trainable=True, name = 'alpha')
    output_layer.add_weight(shape = (), dtype='float32', trainable=True, name = 'beta')
    
    network = keras.Sequential(layers)

    # print(output_layer.trainable_weights)

    return network, output_layer


In [ ]:

network, output_layer = make_network()
alpha = output_layer.trainable_weights[0]
beta = output_layer.trainable_weights[1]

solver = PINNSolver(network, alpha, beta, t_data)
solver.solve_with_TFoptimizer(keras.optimizers.Adam(), t_data, u_data, 5000000)


In [ ]:
network, output_layer = make_network()
network.load_weights("alpha_beta_problem.weights2.h5")
x = np.arange(0, 1, .01, dtype='float32')
prediction = network.predict(x, verbose = 0)

y = np.squeeze(prediction)
# y = list(map(lambda a: a*10,y)) # magnification :(

plt.plot(t_data, u_data) # blue line, actual sine wave
plt.plot(x, y, "r--") # red dashes, neural net prediction
plt.axis((0, 7, -1, 10000))
plt.show()

print(u_data)
print(y)